# Ebay Scraper

In [1]:
import csv
import requests
from lxml import html

In [2]:
def get_page_items(tree):
    container = tree.xpath("//ul[contains(@class, 'srp-results')]")
    if container:
        return container[0].xpath(".//li[contains(@class, 's-item')]")
    else:
        return []

In [3]:
def create_search_record(item):
    title = "".join(item.xpath(".//h3/text()"))
    sub_title = "".join(item.xpath(".//div[@class='s-item__subtitle']/text()"))
    sub_title += " " + "".join(item.xpath(".//div[@class='s-item__subtitle']//span[@class='SECONDARY_INFO']/text()"))
    rating = "".join(item.xpath(".//div[@class='x-star-rating']//span[@class='clipped']/text()"))
    item_price = "".join(item.xpath(".//span[@class='s-item__price']/text()"))
    trending_price = "".join(item.xpath(".//span[@class='s-item__trending-price']/span[@class='STRIKETHROUGH']/text()"))
    item_link = "".join(item.xpath(".//a[@class='s-item__link']/@href"))
    return (title, sub_title, rating, item_price, trending_price, item_link)

In [4]:
def get_next_page(tree):
    return "".join(tree.xpath("//a[@class='pagination__next']/@href"))

In [5]:
def search_ebay(keywords):
    url = "https://www.ebay.com/sch/i.html?&_nkw=" + keywords.replace(" ", "+")
    response = requests.get(url)

    # get first page
    etree = html.fromstring(response.text)
    page_data = []

    # get remaining pages if existing
    while True:
        items = get_page_items(etree)
        if not items:
            break

        for item in items:
            page_data.append(create_search_record(item))

        next_page = get_next_page(etree)
        if not next_page:
            break

        response = requests.get(next_page)
        if response.status_code != 200:
            break

        etree = html.fromstring(response.text)

    return page_data

In [6]:
def save_results(records, filename, save_method='w'):
    with open(filename, save_method, newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Title', 'SubTitle', 'Rating', 'ItemPrice', 'TrendingPrice', 'ItemLink'])
        writer.writerows(records)

In [7]:
# run the program

results = search_ebay('curved monitor')

if results:
    save_results(results, 'curved_monitor.csv')

In [8]:
# show the first few results
results[:3]

[('Deco Gear 35" Curved Ultrawide LED Gaming Monitor Crisp 2560X1080 HD',
  'Authorized Deco Gear Dealer w USA Warranty! Ships Fast! Brand New',
  '5.0 out of 5 stars.',
  '$309.99',
  '$649.99',
  'https://www.ebay.com/itm/Deco-Gear-35-Curved-Ultrawide-LED-Gaming-Monitor-Crisp-2560X1080-HD/333335017371?epid=27029108979&_trkparms=ispr%3D1&hash=item4d9c50b79b:g:tkIAAOSwdtRdhAZG&amdata=enc%3AAQAFAAACcBaobrjLl8XobRIiIML1V4Imu%252Fn%252BzU5L90Z278x5ickkCrLl8erj3ATP5raQxjc%252F%252B0nHYso%252BW3yGz6OulOpZGXxrm2acZsg%252BR%252FpfaZ5jr4KFG1MmLofGglWqwSzLXv0nGcpz0HJwFmLYG%252Fps25XxVC1c9fZ5PryaMCuRseTJpquhvMIipQG8ekBZTPEBJ9V7hV%252BydYzp2QO8cz75GTWFFHGf5NlDoeHryLO60wzrVMPO8JfDOY%252BWa5OEnZNbk2i7EUeJdJdPSJGJo7jM6A%252BHtkXHjjWwPdfWZ0HtLaRcaQXtwAg8XCgZWuNis9X8wSNPFx%252BBj6V17bPh%252BJ6uocmFHynroUu%252BszZmTJnw9ROCoXBQzg3MS%252FR4sBF2WfXRcaZmO2VTHrUCw29UJb7JK2MZiot2NBsV%252F1uIsLfDaEHMAOGZrfngvNvSv1g2owrskN0veQMlNO5aZFubdYore2%252Fk4tlc%252B5Uf3mHKfIMtaIgViclZfC2FRtWHc1XRvzmPsOaGa5RRu00oPBLLhSx